In [1]:
n_scenario = 16

num_scenario = '/' + str(n_scenario) + '_Scenario/'
dir_name = 'coarse_rm_' + str(n_scenario) +'/'

import os
import yaml
import json
import shutil
import numpy as np
from gurobipy import GRB
from utils import prepare_input
from main_model import two_stage_model

In [2]:
with open(r'config.yaml') as file:
    model_params = yaml.load(file, Loader=yaml.FullLoader)

model_params["path_to_input"] = os.path.dirname(os.getcwd()) + '/Data/fixed_reduced_grid' + num_scenario
model_params["path_to_output"] = os.getcwd() + "/output/" + dir_name

if os.path.exists(model_params["path_to_output"]):
    shutil.rmtree(model_params["path_to_output"], ignore_errors=True)
os.mkdir(model_params["path_to_output"])
model_params["input1"], model_params["input2"] = prepare_input(model_params["path_to_input"])

print("MIP-Gap: ", model_params["mip_gap"])
print("Time Limit: ", model_params["time_limit"])
print("Robust Model: ", model_params["robust_flag"])
print("Flexible Generation: ", model_params["flexible_generation"])
print("Objective Type:\t", model_params["set_objective"], "\n")
print("Solver Method:\t", model_params["solver_method"], "\n")

flood_df = model_params["input1"].iloc[:,9:].copy()
flood_df_columns = flood_df.columns
input1_copy = model_params["input1"].drop(flood_df_columns, axis=1).copy()

MIP-Gap:  0.005
Time Limit:  21600
Robust Model:  False
Flexible Generation:  True
Objective Type:	 min 

Solver Method:	 2 



In [3]:
model_params["robust_flag"] = True

## Changes

In [4]:
%%time
budget_vector = [0,10,20,30,40,50,60,70,80,85.625]
base_model = two_stage_model(model_params)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-26
CPU times: user 5.58 s, sys: 109 ms, total: 5.69 s
Wall time: 5.67 s


### Model Run

In [5]:
for budget in budget_vector:
    base_model.budget_ref.rhs = budget*1e6
    base_model.model.setParam("LogFile", model_params["path_to_output"] + str(budget) + "M")
    base_model.model.setParam("MIPGap", model_params["mip_gap"])
    base_model.model.setParam("TimeLimit", model_params["time_limit"])
    base_model.model.setParam("Method", model_params["solver_method"])
    # base_model.model.setParam("MIRCuts", 2)
    # base_model.model.setParam("FlowCoverCuts", 2)
    # base_model.model.setParam("MIPFocus", 3)
    # base_model.model.setParam("ImpliedCuts", 2)
    base_model.model.optimize()
    base_model.model.write(model_params["path_to_output"] + str(budget) + "M_" + "solution.sol")    
    print("\n\n*********************************************")
    print("*********************************************\n\n")
    
    # budget_solution[budget]["objective_value"] = base_model.model.ObjVal
    # for i in base_model.x.keys():
    #     budget_solution[budget]['substation_' + str(i)] = base_model.x[i].X

Set parameter LogFile to value "/Users/ashutoshshukla/Desktop/TwoStageModel/output/coarse_rm_16/0M"
Set parameter MIPGap to value 0.005
Set parameter TimeLimit to value 21600
Set parameter Method to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 211835 rows, 77925 columns and 600776 nonzeros
Model fingerprint: 0xdcb1922f
Variable types: 55985 continuous, 21940 integer (21578 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e-02, 2e+04]
Presolve removed 198923 rows and 60362 columns
Presolve time: 0.34s
Presolved: 12912 rows, 17563 columns, 64813 nonzeros
Variable types: 15428 continuous, 2135 integer (2135 binary)
Found heuristic solution: objective 379.5386330
Root barrier log...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 7.694e+04
 Factor NZ

### Write Solutions

In [6]:
with open(model_params["path_to_output"] + 'model_params.json', 'w') as fp:
    del model_params["input1"]
    del model_params["input2"]
    json.dump(model_params, fp)
    
# with open(model_params["path_to_output"] + 'budget_solution.json', 'w') as fp:
#     json.dump(budget_solution, fp)